1. O que acontecerá na execução dos algoritmos apresentados para a solução do problema da Mochila 0/1 se o número de itens crescer de forma demasiada? Por que isso ocorre? Tente aumentar gradualmente o número de itens e verifique o que ocorre. Relate.


2. Considere que no problema da mochila 0/1 cada um dos itens disponíveis tem um valor e que o objetivo é encher a mochila para maximizar o valor agregado dos ítens colocados na mochila. Refaça a solução em python apresentada anteriormente para resolver este problema.


3. Implemente o algoritmo de agrupamento hierárquico divisivo para dados unidimensionais.


4. Altere o algoritmo de agrupamento hierárquico divisivo para considerar dados multidimensionais numéricos. Indique o que necessita ser feito para aplicar o algoritmo da questão 10 aqui. Qual a maior dificuldade para isso? Proponha uma solução para contornar essa dificuldade. Indique casos em que essa solução pode
não ser satisfatória. Implemente a sua solução.